<a href="https://colab.research.google.com/github/itsjafer/decktor-cli/blob/main/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeckTor Quickstart

Improve your Anki decks using AI directly in your browser. No coding required!


> **⚠️ IMPORTANT:** Your deck must contain **only one type of card** (either all Basic or all Cloze). Select the matching card type below.

In [ ]:
#@title 1. Install DeckTor
#@markdown Run this cell to install the DeckTor package.

from IPython.display import clear_output

!pip install git+https://github.com/itsjafer/decktor-cli.git
clear_output()
print("✅ Installation complete!")

In [ ]:
#@title 2. Choose Model & Enter API Key
#@markdown Select a model provider. The **Free** option requires no API key.
#@markdown 
#@markdown If you select a paid provider, enter the corresponding API key below.

import os

model = "GPT4Free (Free)" #@param ["GPT4Free (Free)", "Gemini 2.5 Flash Lite", "Gemini 2.5 Flash", "GPT-5 Mini", "GPT-5", "Claude Haiku 4.5", "Claude Sonnet 4.5"]
API_KEY = "" #@param {type:"string"}

# Determine which API key env var to set based on model
if model == "GPT4Free (Free)":
    print("✅ Using GPT4Free (Free) - no API key needed!")
elif model.startswith("Gemini"):
    if API_KEY:
        os.environ["GOOGLE_API_KEY"] = API_KEY
        print("✅ Gemini API Key configured!")
    else:
        print("❌ Please enter your Google API Key above (get one at https://aistudio.google.com/api-keys)")
elif model.startswith("GPT-5"):
    if API_KEY:
        os.environ["OPENAI_API_KEY"] = API_KEY
        print("✅ OpenAI API Key configured!")
    else:
        print("❌ Please enter your OpenAI API Key above (get one at https://platform.openai.com/api-keys)")
elif model.startswith("Claude"):
    if API_KEY:
        os.environ["ANTHROPIC_API_KEY"] = API_KEY
        print("✅ Anthropic API Key configured!")
    else:
        print("❌ Please enter your Anthropic API Key above (get one at https://console.anthropic.com/settings/keys)")

In [ ]:
#@title 3. Upload Your Deck
#@markdown Click the play button to upload your `.apkg` file.

from google.colab import files

uploaded = files.upload()
input_filename = next(iter(uploaded))
print(f"\n✅ Uploaded: {input_filename}")

In [ ]:
#@title 4. Run DeckTor
#@markdown Select your card type and processing options, then run.
#@markdown 
#@markdown **Card Type:** Select the type that matches ALL cards in your deck.
#@markdown - **Basic (Front/Back):** Standard question/answer cards
#@markdown - **Cloze:** Cards with {{c1::cloze deletions}}

import os
import subprocess

output_filename = "improved_" + input_filename

# Configuration Options
card_type = "Basic (Front/Back)" #@param ["Basic (Front/Back)", "Cloze"]
batch_size = 10 #@param {type:"integer"}
limit = 10 #@param {type:"integer"}

# Get the prompt file path based on card type
result = subprocess.run(['python', '-c', 'import decktor; import os; print(os.path.dirname(decktor.__file__))'], capture_output=True, text=True)
decktor_path = result.stdout.strip()

if card_type == "Basic (Front/Back)":
    prompt_file = os.path.join(decktor_path, 'prompts', 'default.txt')
else:
    prompt_file = os.path.join(decktor_path, 'prompts', 'cloze.txt')

# Construct command
cmd = f'decktor process "{input_filename}" "{output_filename}" --model "{model}" --batch-size {batch_size} --prompt "{prompt_file}"'

if limit > 0:
    cmd += f' --limit {limit}'

print(f"🤖 Model: {model}")
print(f"📝 Card Type: {card_type}")
print(f"Running: {cmd}\n")
!{cmd}

print(f"\n✅ Done! Saved to: {output_filename}")

In [ ]:
#@title 5. Download Result
from google.colab import files
files.download(output_filename)